Pipeline for Extracting Keywords from the dataset

In [2]:
import pandas as pd
import spacy
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import numpy as np
#import nltk
#nltk.download('punkt')


In [3]:
nlp = spacy.load("en_core_web_sm")

path = "/Users/ethanvirtudazo/Desktop/DS105_Dataset/6622_jobs.xls"
df_import = pd.read_excel(path)

df_import.shape

(6623, 11)

In [84]:
df1 = df_import.iloc[:, 1:5]
df1.head()

df2 = df_import.iloc[:, [7,9,10]]
df2.head()

pdList = [df1,df2]
df = pd.concat(pdList,axis=1)
df.head()

df.shape

types = df.dtypes
print(types)

df_title = df.iloc[:,0]
df_text_both = df.iloc[:, [0,1]]
df_text = df.iloc[:, [1]]

df_text_both.head()


title                  object
details                object
deadline       datetime64[ns]
opport_type            object
location               object
company                object
links                  object
dtype: object


,title,details
0,Rothschild & Co - Private Equity Long-Term Int...,This London-based 6-month internship is an exc...
1,2023 HSBC Global Graduate Programme (Hong Kong...,You’re excited about starting your career and ...
2,2023 HSBC Global Internship Programme (Hong Ko...,You’re excited about starting your career and ...
3,"Graduate Training Scheme, Capital Markets",Graduate Training Scheme – LondonGreySpark Par...
4,6-Months Internship – Sell-side Tech M&A,"At IPTP, we understand software from decades o..."


In [5]:
df_text.head()

,details
0,This London-based 6-month internship is an exc...
1,You’re excited about starting your career and ...
2,You’re excited about starting your career and ...
3,Graduate Training Scheme – LondonGreySpark Par...
4,"At IPTP, we understand software from decades o..."


In [6]:
ar_text = df_text.to_numpy()
ar_text = ar_text[0:101]

In [35]:
nlp = spacy.load("en_core_web_sm")

# tokens contains a list of list of tokens
tokens = np.array([[token.text for token in nlp(sent[0])] for sent in ar_text])

/Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  after removing the cwd from sys.path.


In [51]:
docs1 = [' '.join(doc) for doc in ar_text]

filtered_docs = list([[token.text for token in nlp(doc) if token.pos_ in ['PROPN', 'NOUN','ADJ']] for doc in docs1])

In [93]:
# POS TEST
filtered_docs

fil_docs = [' '.join(doc) for doc in filtered_docs]

docs = list(nlp.pipe(fil_docs))

In [78]:
# POS TEST

pos_tags = list([[token.pos_ for token in doc] for doc in docs])
len(pos_tags)
#pos_tags = np.array(pos_tags)

101

In [88]:
# POS TEST

df_tk = pd.DataFrame(filtered_docs)
df_ps = pd.DataFrame(pos_tags)
len(df_tk)

101

In [97]:
rend_doc = docs[0]

In [95]:
rend_doc

'London month internship exciting opportunity Rothschild Co European corporate private equity business Rothschild Co European corporate private equity business strategies Arrows Principal Investments FAPI Arrows Growth Capital FAGC Arrows Long Term Fund FALT FAPI c. latest fund flagship corporate private equity vehicle mid - cap companies FAGC c. growth investments small cap buy outs FALT investments larger companies option longer term holding periods funds investments Western Europe FALT North America main verticals Data Software Healthcare Technology Business Services respective teams London Paris Luxembourg c. investment professionals countries Responsibilities industries potential targets investments sources information Identifying investment opportunities financial models investment committee memos standard financial models deal execution processes monitoring portfolio companies point view value actionable opportunities Fund risk return criteria team co - due diligence transaction

In [98]:
from spacy import displacy
displacy.render(rend_doc,style="ent")

In [89]:
# CREATING DATAFRAME FOR R
df_title = df_title[0:101]

In [90]:
# CREATING DATAFRAME FOR R

df_title_tks = pd.concat([df_title, df_tk], axis = 1)
df_title_tks

,title,0,1,2,3,4,5,6,7,8,...,282,283,284,285,286,287,288,289,290,291
0,Rothschild & Co - Private Equity Long-Term Int...,London,month,internship,exciting,opportunity,Rothschild,Co,European,corporate,...,None,None,None,None,None,None,None,None,None,None
1,2023 HSBC Global Graduate Programme (Hong Kong...,excited,career,many,paths,possibilities,global,bank,inclusive,culture,...,None,None,None,None,None,None,None,None,None,None
2,2023 HSBC Global Internship Programme (Hong Ko...,excited,career,many,paths,possibilities,global,bank,inclusive,culture,...,None,None,None,None,None,None,None,None,None,None
3,"Graduate Training Scheme, Capital Markets",Graduate,Training,Scheme,LondonGreySpark,Partners,Graduate,Training,Scheme,London,...,None,None,None,None,None,None,None,None,None,None
4,6-Months Internship – Sell-side Tech M&A,IPTP,software,decades,deep,experience,technology,sector,strategic,commercial,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,M&A Analyst Intern,MAJOR,RESPONSIBILITIESGather,financial,operating,information,companies,industries,governmentsCreate,financial,...,None,None,None,None,None,None,None,None,None,None
97,12 Month Internship - Financial Crimes and San...,Job,summaryFinancial,Crime,Financial,Security,Credit,Agricole,vacancy,Central,...,None,None,None,None,None,None,None,None,None,None
98,Investment Associate - Fixed Income,Position,OverviewPutnam,energetic,curious,individuals,eager,bond,dynamic,environment,...,None,None,None,None,None,None,None,None,None,None
99,12 Month Internship - Central Compliance,SummaryThe,Central,Compliance,team,responsible,transversal,compliance,regulatory,matters,...,None,None,None,None,None,None,None,None,None,None
